<h1 align="center">Theano</h1>

In [ ]:
!pip install numpy matplotlib   
!pip install --upgrade https://github.com/Theano/Theano/archive/master.zip
!pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip

### Разминка

In [ ]:
import theano
import theano.tensor as T

%pylab inline

#### будущий параметр функции -- символьная переменная

In [ ]:
N = T.scalar('a dimension', dtype='float32')

#### рецепт получения квадрата -- орперации над символьными переменным

In [ ]:
result = T.power(N, 2)

#### theano.grad(cost, wrt)

In [ ]:
grad_result = theano.grad(result, N) 

#### компиляция функции "получения квадрата"

In [ ]:
sq_function = theano.function(inputs=[N], outputs=result)
gr_function = theano.function(inputs=[N], outputs=grad_result)

#### применение функции

In [ ]:
# Заводим np.array x
xv = np.arange(-10, 10)

# Применяем функцию к каждому x
val = map(float, [sq_function(x) for x in xv])

# Посичтаем градиент в кажой точке
grad = map(float, [gr_function(x) for x in xv])

### Что мы увидим если нарисуем функцию и градиент?

In [ ]:
pylab.plot(xv, val, label='x*x')
pylab.plot(xv, grad, label='d x*x / dx')
pylab.legend()

<h1 align="center">Lasagne</h1>

* lasagne - это библиотека для написания нейронок произвольной формы на theano
* В качестве демо-задачи выберем то же распознавание чисел, но на большем масштабе задачи, картинки 28x28, 10 цифр

In [ ]:
from mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

print 'X размера', X_train.shape, 'y размера', y_train.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(20, 20))

for i, ax in enumerate(axes):
    ax.imshow(X_train[i, 0], cmap='gray')

Давайте посмотрим на DenseLayer в lasagne
- http://lasagne.readthedocs.io/en/latest/modules/layers/dense.html
- https://github.com/Lasagne/Lasagne/blob/master/lasagne/layers/dense.py#L16-L124 
- Весь содаржательный код тут https://github.com/Lasagne/Lasagne/blob/master/lasagne/layers/dense.py#L121 

In [ ]:
import lasagne
from lasagne import init
from theano import tensor as T
from lasagne.nonlinearities import softmax

X, y = T.tensor4('X'), T.vector('y', 'int32')

Так задаётся архитектура нейронки

In [ ]:
#входной слой (вспомогательный)
net = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=X)

net = lasagne.layers.Conv2DLayer(net, 15, 28, pad='valid', W=init.Constant()) # сверточный слой
net = lasagne.layers.Conv2DLayer(net, 10,  2, pad='full', W=init.Constant())  # сверточный слой

net = lasagne.layers.DenseLayer(net, num_units=500) # полносвязный слой
net = lasagne.layers.DropoutLayer(net, 1.0)         # регуляризатор
net = lasagne.layers.DenseLayer(net, num_units=200) # полносвязный слой

net = lasagne.layers.DenseLayer(net, num_units=10)  # полносвязный слой

In [ ]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(net)

In [ ]:
#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(net)
print all_weights

In [ ]:
#функция ошибки и точности будет прямо внутри
loss = lasagne.objectives.categorical_accuracy(y_predicted, y).mean()
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, y).mean()

In [ ]:
#сразу посчитать словарь обновлённых значений с шагом по градиенту, как раньше
updates = lasagne.updates.momentum(loss, all_weights, learning_rate=1.0, momentum=1.5)

In [ ]:
#функция, делает updates и возвращащет значение функции потерь и точности
train_fun = theano.function([X, y], [loss, accuracy], updates=updates)
accuracy_fun = theano.function([X, y], accuracy) # точность без обновления весов, для теста

# Процесс обучения

In [ ]:
import time 
from mnist import iterate_minibatches

num_epochs = 5  #количество проходов по данным
batch_size = 50 #размер мини-батча

for epoch in range(num_epochs):
    train_err, train_acc, train_batches, start_time = 0, 0, 0, time.time()
    for inputs, targets in iterate_minibatches(X_train, y_train, batch_size):
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    val_acc, val_batches = 0, 0
    for inputs, targets in iterate_minibatches(X_test, y_test, batch_size):
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    print "Epoch %s of %s took %.3f s" % (epoch + 1, num_epochs, time.time() - start_time)
    print " train loss:\t %.3f"  % (train_err / train_batches)
    print " train  acc:\t %.3f" % (train_acc * 100 / train_batches), '%'
    print " test   acc:\t %.3f"  % (val_acc  * 100 / val_batches), '%'
    print

In [ ]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results: \n test accuracy:\t\t{:.2f} %".format(test_acc / test_batches * 100))

# Ансамблирование с DropOut

In [ ]:
#предсказание нейронки (theano-преобразование)
y_predicted = T.mean([lasagne.layers.get_output(net, deterministic=False) for i in range(10)], axis=0)
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, y).mean()
accuracy_fun = theano.function([X, y], accuracy) # точность без обновления весов, для теста

In [ ]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results: \n test accuracy:\t\t{:.2f} %".format(test_acc / test_batches * 100))